In [1]:
import equinox as eqx
import gym
import jax
import jax.numpy as jnp
import jax.random as jrandom
import numpy as np
import optax
import sys
import timeit
import wandb

from argparse import Namespace
from functools import partial
from jax import grad, jit, vmap
from typing import Sequence

from jax_learning.agents.rl_agents import EpsilonGreedyAgent
from jax_learning.buffers.ram_buffers import NextStateNumPyBuffer
from jax_learning.buffers.utils import batch_flatten, to_jnp
from jax_learning.common import init_wandb
from jax_learning.constants import DISCRETE
from jax_learning.learners.q_learning import QLearning
from jax_learning.models.q_functions import SoftmaxQ, MLPQ
from jax_learning.rl_utils import interact, evaluate

In [2]:
init_wandb(project="test_jax_rl", group="cartpole-dqn_test", mode="disabled")

/Users/chanb/research/personal/jax_learning/.venv/lib/python3.10/site-packages/notebook/utils.py:280: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  return LooseVersion(v) >= LooseVersion(check)


In [3]:
cfg_dict = {
    # Environment setup
    "env": "CartPole-v1",
    "seed": 0,
    "render": False,
    
    # Experiment progress
    "load_step": 0,
    "log_interval": 5000,
    
    # Learning hyperparameters
    "max_timesteps": 1000000,
    "buffer_size": 1000000,
    "buffer_warmup": 1000,
    "num_gradient_steps": 1,
    "batch_size": 64,
    "lr": 3e-4,
    "max_grad_norm": 10.,
    "gamma": 0.99,
    "update_frequency": 4,
    "target_update_frequency": 1,
    "tau": 0.005, # This is for polyak averaging of target network
    "omega": 1.0, # This is for residual gradient: 1 for semi-gradient
    
    # Epsilon greedy hyperparameters
    "init_eps": 1.0,
    "min_eps": 0.02,
    "eps_decay": 0.9999,
    "eps_warmup": 1000,
    
    # Normalization
    "normalize_obs": False,
    "normalize_value": False,
    
    # Model architecture
    "hidden_dim": 64,
    "num_hidden": 1,
    
    # Evaluation
    "evaluation_frequency": 5000,
    "eval_cfg": {
        "num_episodes": 10,
        "seed": 1,
        "render": True,
    }
}
cfg = Namespace(**cfg_dict)
eval_cfg = Namespace(**cfg.eval_cfg)
wandb.config = cfg_dict

In [4]:
np.random.seed(cfg.seed)

In [5]:
env = gym.make(cfg.env)

/Users/chanb/research/personal/jax_learning/.venv/lib/python3.10/site-packages/gym/core.py:329: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/Users/chanb/research/personal/jax_learning/.venv/lib/python3.10/site-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [6]:
cfg.obs_dim = env.observation_space.shape
cfg.act_dim = (env.action_space.n,)
cfg.action_space = DISCRETE

In [7]:
cfg.h_state_dim = (1,)
cfg.rew_dim = (1,)

In [8]:
cfg.buffer_rng = np.random.RandomState(cfg.seed)
cfg.env_rng = np.random.RandomState(cfg.seed)
cfg.agent_key, cfg.model_key = jrandom.split(jrandom.PRNGKey(cfg.seed), num=2)
eval_cfg.env_rng = np.random.RandomState(eval_cfg.seed)
cfg.evaluation_cfg = eval_cfg

In [9]:
cfg

Namespace(env='CartPole-v1', seed=0, render=False, load_step=0, log_interval=5000, max_timesteps=1000000, buffer_size=1000000, buffer_warmup=1000, num_gradient_steps=1, batch_size=64, lr=0.0003, max_grad_norm=10.0, gamma=0.99, update_frequency=4, target_update_frequency=1, tau=0.005, omega=1.0, init_eps=1.0, min_eps=0.02, eps_decay=0.9999, eps_warmup=1000, normalize_obs=False, normalize_value=False, hidden_dim=64, num_hidden=1, evaluation_frequency=5000, eval_cfg={'num_episodes': 10, 'seed': 1, 'render': True}, obs_dim=(4,), act_dim=(2,), action_space='discrete', h_state_dim=(1,), rew_dim=(1,), buffer_rng=RandomState(MT19937) at 0x286894E40, env_rng=RandomState(MT19937) at 0x286895140, agent_key=DeviceArray([4146024105,  967050713], dtype=uint32), model_key=DeviceArray([2718843009, 1272950319], dtype=uint32), evaluation_cfg=Namespace(num_episodes=10, seed=1, render=True, env_rng=RandomState(MT19937) at 0x286895040))

In [10]:
Q = "q"

buffer = NextStateNumPyBuffer(
    buffer_size=cfg.buffer_size,
    obs_dim=cfg.obs_dim,
    h_state_dim=cfg.h_state_dim,
    act_dim=(1,) if cfg.action_space == DISCRETE else cfg.act_dim,
    rew_dim=cfg.rew_dim,
    rng=cfg.buffer_rng,
)

model = {
    Q: SoftmaxQ(q_function=MLPQ(in_dim=cfg.obs_dim,
                                out_dim=cfg.act_dim,
                                hidden_dim=cfg.hidden_dim,
                                num_hidden=cfg.num_hidden,
                                key=cfg.model_key)),
}

target_model = {
    Q: SoftmaxQ(q_function=MLPQ(in_dim=cfg.obs_dim,
                                out_dim=cfg.act_dim,
                                hidden_dim=cfg.hidden_dim,
                                num_hidden=cfg.num_hidden,
                                key=cfg.model_key)),
}

opt_transforms = [
    optax.scale_by_rms(),
    optax.scale(-cfg.lr)
]
if cfg.max_grad_norm:
    opt_transforms.insert(0, optax.clip_by_global_norm(cfg.max_grad_norm))
opt = {
    Q: optax.chain(*opt_transforms)
}

learner = QLearning(model=model,
                    target_model=target_model,
                    opt=opt,
                    buffer=buffer,
                    cfg=cfg)

agent = EpsilonGreedyAgent(model=model,
                           model_key=Q,
                           buffer=buffer,
                           learner=learner,
                           init_eps=cfg.init_eps,
                           min_eps=cfg.min_eps,
                           eps_decay=cfg.eps_decay,
                           eps_warmup=cfg.eps_warmup,
                           action_space=DISCRETE,
                           action_dim=cfg.act_dim[0],
                           key=cfg.agent_key)

In [11]:
%wandb

In [12]:
interact(env, agent, cfg)

Epoch 1 ==================================================
Current return (episode: 257, is finished: False) with length 0: 0.0
[3.4384317 3.5290809]


/Users/chanb/research/personal/jax_learning/.venv/lib/python3.10/site-packages/gym/core.py:57: DeprecationWarning: WARN: You are calling render method, but you didn't specified the argument render_mode at environment initialization. To maintain backward compatibility, the environment will render in human mode.
If you want to render in human mode, initialize the environment in this way: gym.make('EnvName', render_mode='human') and don't call the render method.
See here for more information: https://www.gymlibrary.ml/content/api/
  deprecation(


DependencyNotInstalled: pygame is not installed, run `pip install gym[classic_control]`

In [ ]:
wandb.finish()